In [1]:
import pandas as pd
certificates = pd.read_csv('C:\\Users\\Dell\\OneDrive\\DataR Labs\\Resume_Parser\\certificates.csv')
education_df = pd.read_csv('EducationWords.csv')
certificates = pd.read_csv('C:\\Users\\Dell\\OneDrive\\DataR Labs\\Resume_Parser\\certificates.csv')
import nltk
from nltk.util import ngrams
import itertools

In [2]:
g_df1 = pd.read_csv('C:\\Users\\Dell\\OneDrive\\Self\\Data_RP\\glassdoor_1.csv')
g_df2 = pd.read_csv('C:\\Users\\Dell\\OneDrive\\Self\\Data_RP\\glassdoor_2.csv')
g_df3 = pd.read_csv('C:\\Users\\Dell\\OneDrive\\Self\\Data_RP\\glassdoor_3.csv')
g_df4 = pd.read_csv('C:\\Users\\Dell\\OneDrive\\Self\\Data_RP\\glassdoor_4.csv')
g_df5 = pd.read_csv('C:\\Users\\Dell\\OneDrive\\Self\\Data_RP\\glassdoor_5.csv')
g_df6 = pd.read_csv('C:\\Users\\Dell\\OneDrive\\Self\\Data_RP\\glassdoor_6.csv')
g_df7 = pd.read_csv('C:\\Users\\Dell\\OneDrive\\Self\\Data_RP\\glassdoor_7.csv')
g_df8 = pd.read_csv('C:\\Users\\Dell\\OneDrive\\Self\\Data_RP\\glassdoor_8.csv')

m_df1 = pd.read_csv('C:\\Users\\Dell\\OneDrive\\Self\\Data_RP\\moster_1.csv')
m_df2 = pd.read_csv('C:\\Users\\Dell\\OneDrive\\Self\\Data_RP\\moster_2.csv')
m_df3 = pd.read_csv('C:\\Users\\Dell\\OneDrive\\Self\\Data_RP\\moster_3.csv')
m_df4 = pd.read_csv('C:\\Users\\Dell\\OneDrive\\Self\\Data_RP\\moster_4.csv')
m_df5 = pd.read_csv('C:\\Users\\Dell\\OneDrive\\Self\\Data_RP\\moster_5.csv')

In [4]:
g_df6 = g_df6.drop(['Salary Estimate'],axis=1)
g_df1 = g_df1.drop(['Salary Estimate'],axis=1)

In [5]:
# Function to generate n-grams from sentences.
def extract_ngrams(data):
    l = []
    for i in range(1,6):
        n_grams = ngrams(nltk.word_tokenize(data), i)
        l.append( [' '.join(grams) for grams in n_grams])
    flat_ls = list(itertools.chain(*l))
    return (flat_ls)

In [65]:
import re
import unicodedata
import spacy
from spacy.lang.en.stop_words import STOP_WORDS
from nltk.util import ngrams


def remove_accented_chars(x):
    """The function changes the accented characters into their equivalent normal form,
    to do so, normalize function with 'NFKD' is used which replaces the compatibility characters into
    theri euivalent
    
  param x(str): the sentence in which accented characters are to be detected and removes
  return x(str): sentence with accented characters replaced by their equivalent"""
    
    x = unicodedata.normalize('NFKD', x).encode('ascii', 'ignore').decode('utf-8', 'ignore')
    return x


def preprocess(df,d):
    """Preprocesses the given document by applying the following functionalities
  lower: lowers all the characters for uniformity
  remove special characters: using regex, removes all the punctuations etc
  remove space: removes trailing spaces and extra spaces between words
  remove accented characters: change accented characters to its normal equivalent
  remove stop words"""
  
    df[d] = df[d].apply(lambda x: x.lower())
    df[d] = df[d].apply(lambda x: re.sub('[^A-Z a-z 0-9-]+', '', x))
    df[d] = df[d].apply(lambda x: " ".join(x.split()))
    df[d] = df[d].apply(lambda x: remove_accented_chars(x))
    df[d] = df[d].apply(lambda x: " ".join([t for t in x.split() if t not in STOP_WORDS]))


def salary_extract(x):
    #change to string
    if x == -1:
        return 'Missing Value'
    return x.astype(str)    
    
def get_education(x):
    '''
    param
      -x : Text field from where education is to be extracted
    return
      -ed : Education as matched from the education list
      '''
    
    education_df = pd.read_csv('EducationWords.csv')
    education_words = education_df['Education'].tolist()
    education_words = [item.lower() for item in education_words]
    #print(education_words)
    ed = ''
    for word in education_words:
        #print(x)
        #print(word)
        if word in x:
            #print(word)
            ed = ed + "|" + word
    
    if ed!='':
        return ed
    else:
        return ('-1')
    
def get_experience(x):
    """
    param
     -x : Textfield from where experience is to be extracted
    return
     -item : Returns year of experience if available, or returns -1 if not available 
    """
    
    exp = re.findall('\\d+\\s+(?:months?|years?)',x)
    if exp:
        for item in exp:
            return item
    else:
        return ('-1')    


def get_responsibility(x):
    """
    param
     -x : Textfield from where responsibility is to be extracted
    return
     -item : Returns responsibility if available, or returns -1 if not available 
    """
    
    try:
        pos = x.index("responsibilities")
    except ValueError:
        return (x)
    return (x[pos:pos+600])

def get_skills(x):
    """
    param
     -x : Textfield from where responsibility is to be extracted
    return
     -item : Returns responsibility if available, or returns -1 if not available 
    """
    
    try:
        pos = x.index("skills")
    except ValueError:
        return (x)
    return (x[pos:pos+600])


def get_certificate(x):

    """
    Matched field with with certificate dictionary and returns certificates if present in feild
    
    param
     -x : Textfield from where certificate is to be extracted
    return
     -item : Returns certificate if available, or returns -1 if not available 
    """
    
    #get certificates
    certificate_list = certificates['Certificates']

    #new list to contain the elements
    newList = []
    #get elements
    for element in certificate_list:
        newList.extend(element.split(','))

    res = []
    #remove duplicates
    [res.append(x) for x in newList if x not in res]

    #remove spaces
    res = [elem for elem in res if elem.strip()]

    #remove spaces in words
    res = [elem.strip() for elem in res]

    #normalize
    certificate_dictionary = [elem.lower() for elem in res]
    #print(res)
                          
    cert = ''
    
    for word in certificate_dictionary:
        if word in x:
            cert = cert + "|" + word
    if cert == '':
        return ('-1')
    
    else:
        return cert

def get_language(x):

    """
    Matched field with with language dictionary and returns languages if present in feild
    
    param
     -x : Textfield from where language is to be extracted
    return
     -item : Returns language if available, or returns -1 if not available 
    """
    
    language_dictionary = ['english',
                           'french',
                           'spanish',
                           'chinese',
                           'german',
                           'mandarin',
                           'japanese',
                           'russian']
    lang = ''
    
    for word in language_dictionary:
        if word in x:
            lang = lang + "|" + word
    if lang == '':
        return ('-1')
    else:
        return lang

In [7]:
def extraction(df):
    df['Full Description'] = df['Full Description'].astype(pd.StringDtype())
    preprocess(df,'Full Description')
    df['Skills'] = df['Full Description'].apply(get_responsibility)
    df['Responsibilities'] = df['Full Description'].apply(get_responsibility)
    df['education'] = df['Full Description'].apply(get_education)
    df['Experience in years'] = df['Full Description'].apply(get_experience)
    df['Experience in years'] = df['Experience in years'].map(lambda x : x.split(' ')[0])
    df['bow'] = df['Full Description'].apply(extract_ngrams)
    df['Certification'] = df['bow'].apply(get_certificate)
    df['Foreign Language'] = df['Full Description'].apply(get_language)

In [8]:
def clean_df(df):
    url = df['URL']
    df = df.drop(['URL','Unnamed: 0','bow'],axis=1)
    df['Link'] = url
    df = df.replace(['-1'],'No Value')
    return df

In [103]:
def clean_experience(x):
    if x == 'No Value':
        return x
    #print(x)
    x = int(x)
    #print(type(x))
    if x>30:
        return 'No Value'
    else:
        return x

In [21]:
extraction(g_df1)
print("Glassdoor 1-")

extraction(g_df2)
print("Glassdoor 2-")

extraction(g_df3)
print("Glassdoor 3-")

extraction(g_df4)
print("Glassdoor 4-")

extraction(g_df5)
print("Glassdoor 5-")

extraction(g_df6)
print("Glassdoor 6-")

extraction(g_df7)
print("Glassdoor 7-")

extraction(g_df8)
print("Glassdoor 8-")

extraction(m_df1)
print("Monster 1-")

extraction(m_df2)
print("Monster 2-")

extraction(m_df3)
print("Monster 3-")

extraction(m_df4)
print("Monster 4-")

extraction(m_df5)
print("Monster 5-")

Glassdoor 1-
Glassdoor 2-
Glassdoor 3-
Glassdoor 4-
Glassdoor 5-
Glassdoor 6-
Glassdoor 7-
Glassdoor 8-
Monster 1-
Monster 2-
Monster 3-
Monster 4-
Monster 5-


In [22]:
g_df2 = clean_df(g_df2)
g_df3 = clean_df(g_df3)
g_df4 = clean_df(g_df4)
g_df5 = clean_df(g_df5)
g_df7 = clean_df(g_df7)
g_df8 = clean_df(g_df8)

m_df1 = clean_df(m_df1)
m_df2 = clean_df(m_df2)
m_df3 = clean_df(m_df3)
m_df4 = clean_df(m_df4)
m_df5 = clean_df(m_df5)

In [23]:
glassdoor = pd.concat([g_df4, g_df2, g_df3, g_df5, g_df6, g_df7,g_df8], ignore_index=True)
#glassdoor = pd.concat([g_df8], ignore_index=True)

In [24]:
glassdoor = glassdoor.drop(['Salary Estimate','Unnamed: 0'],axis=1)

In [25]:
monster = pd.concat([m_df1, m_df2, m_df3,m_df4,m_df5], ignore_index=True)
#monster = pd.concat([m_df4], ignore_index=True)

In [26]:
monster.fillna('No Value',inplace=True)

In [42]:
glassdoor.fillna('No Value',inplace=True)

In [27]:
glassdoor.shape

(5902, 13)

In [44]:
glassdoor.head()

,Job Title,Searched Job Title,Company Name,Full Description,Location,Job Function,Skills,Responsibilities,education,Experience in years,Certification,Foreign Language,Link
0,Network Technician,Network Technician,Interactive Data Systems Limited,1 installing equipment data communications net...,India,Network Technician,1 installing equipment data communications net...,1 installing equipment data communications net...,No Value,No Value,No Value,No Value,https://www.glassdoor.com/partner/jobListing.h...
1,Network Technician,Network Technician,Interactive Data Systems Limited,1 installing equipment data communications net...,India,No Value,1 installing equipment data communications net...,1 installing equipment data communications net...,No Value,No Value,No Value,No Value,https://www.glassdoor.com/partner/jobListing.h...
2,Senior Network Technician,Senior Network Technician,Apar Consultancy Services,required skillsjunipersecuritynetwork fwpalo a...,India,Network Technician,required skillsjunipersecuritynetwork fwpalo a...,required skillsjunipersecuritynetwork fwpalo a...,No Value,No Value,No Value,No Value,No Value
3,Hardware Networking Specialist,Hardware Networking Specialist,Niyatee Foundation,job description respond hardware related mails...,India,Network Specialist,job description respond hardware related mails...,job description respond hardware related mails...,No Value,No Value,No Value,No Value,https://www.glassdoor.com/partner/jobListing.h...
4,Online Marketer,Online Marketer,Dewelopers,job descriptionoptimizing web content seo stra...,New Delhi,No Value,job descriptionoptimizing web content seo stra...,job descriptionoptimizing web content seo stra...,No Value,No Value,No Value,No Value,No Value


In [29]:
monster.shape

(32150, 12)

In [30]:
monster.head()

,Job Title,Searched Job Title,Company Name,Full Description,Location,Skills,Responsibilities,education,Experience in years,Certification,Foreign Language,Link
0,Application Engineer,Application Developer,Rocket Auto LLC,minimum qualifications bachelors degree comput...,"Detroit, MI 48226",responsibilitiesdesign develop test implement ...,responsibilitiesdesign develop test implement ...,No Value,3,No Value,No Value,https://www.monster.com/job-openings/applicati...
1,Application Developer,Application Developer,Randstad US,job summarythis opportunity mid level applicat...,"Plymouth, MI 48170",responsibilitieswrite requirements test plans ...,responsibilitieswrite requirements test plans ...,No Value,100000,No Value,No Value,https://www.monster.com/job-openings/applicati...
2,Sr Windows App Developer,Application Developer,Hypori,us-tx-austincategory information technologyemp...,"Austin, TX 78759",responsibilities participate complete software...,responsibilities participate complete software...,No Value,7,No Value,No Value,https://www.monster.com/job-openings/sr-window...
3,"Application Engineer, Asset Management",Application Developer,Rock Central,preferred qualifications experience taking pro...,"Remote, MI 99999",responsibilities design develop test implement...,responsibilities design develop test implement...,No Value,2,No Value,No Value,https://www.monster.com/job-openings/applicati...
4,COBOL Application Developer,Application Developer,T.E.S. Employment Solution,cobol application developer little rock arkans...,"Little Rock, AR 72201",responsibilitiesfollowing sdlc analysis design...,responsibilitiesfollowing sdlc analysis design...,No Value,2,No Value,No Value,https://www.monster.com/job-openings/cobol-app...


In [68]:
glassdoor = glassdoor.drop(['education'],axis=1)

In [69]:
monster['Education'] = monster['Full Description'].apply(get_education)

In [70]:
monster = monster.drop(['education'],axis=1)

In [106]:
df.to_csv('monster_jobs_scrapped.csv')
df.to_excel("monster_jobs.xlsx")

In [98]:
df.to_csv('glassdoor_jobs_scrapped_.csv')
#glassdoor.to_excel("glassdoor_jobs_.xlsx")

In [73]:
import pandas as pd

In [99]:
df=pd.read_csv('monster_jobs_scrapped.csv')

In [100]:
df.fillna('No Value',inplace=True)

In [101]:
df['Experience in years'] = df['Experience in years'].astype(pd.StringDtype())
#df['Experience in years'] = df['Experience in years'].astype(int)

In [102]:
df.info()

<class 'pandas.core.frame.DataFrame'>
RangeIndex: 32150 entries, 0 to 32149
Data columns (total 13 columns):
 #   Column               Non-Null Count  Dtype 
---  ------               --------------  ----- 
 0   Unnamed: 0           32150 non-null  int64 
 1   Job Title            32150 non-null  object
 2   Searched Job Title   32150 non-null  object
 3   Company Name         32150 non-null  object
 4   Full Description     32150 non-null  object
 5   Location             32150 non-null  object
 6   Skills               32150 non-null  object
 7   Responsibilities     32150 non-null  object
 8   Experience in years  32150 non-null  string
 9   Certification        32150 non-null  object
 10  Foreign Language     32150 non-null  object
 11  Link                 32150 non-null  object
 12  Education            32150 non-null  object
dtypes: int64(1), object(11), string(1)
memory usage: 3.2+ MB


In [104]:
df['Experience in years'] = df['Experience in years'].apply(clean_experience) 

In [105]:
df

,Unnamed: 0,Job Title,Searched Job Title,Company Name,Full Description,Location,Skills,Responsibilities,Experience in years,Certification,Foreign Language,Link,Education
0,0,Application Engineer,Application Developer,Rocket Auto LLC,minimum qualifications bachelors degree comput...,"Detroit, MI 48226",responsibilitiesdesign develop test implement ...,responsibilitiesdesign develop test implement ...,3,No Value,No Value,https://www.monster.com/job-openings/applicati...,|ca|m a|be|me
1,1,Application Developer,Application Developer,Randstad US,job summarythis opportunity mid level applicat...,"Plymouth, MI 48170",responsibilitieswrite requirements test plans ...,responsibilitieswrite requirements test plans ...,No Value,No Value,No Value,https://www.monster.com/job-openings/applicati...,|ca|b a|me
2,2,Sr Windows App Developer,Application Developer,Hypori,us-tx-austincategory information technologyemp...,"Austin, TX 78759",responsibilities participate complete software...,responsibilities participate complete software...,7,No Value,No Value,https://www.monster.com/job-openings/sr-window...,|ca|be|me
3,3,"Application Engineer, Asset Management",Application Developer,Rock Central,preferred qualifications experience taking pro...,"Remote, MI 99999",responsibilities design develop test implement...,responsibilities design develop test implement...,2,No Value,No Value,https://www.monster.com/job-openings/applicati...,|ca|m a|be|me
4,4,COBOL Application Developer,Application Developer,T.E.S. Employment Solution,cobol application developer little rock arkans...,"Little Rock, AR 72201",responsibilitiesfollowing sdlc analysis design...,responsibilitiesfollowing sdlc analysis design...,2,No Value,No Value,https://www.monster.com/job-openings/cobol-app...,|ca|m a|be|me
...,...,...,...,...,...,...,...,...,...,...,...,...,...
32145,32145,Information Technology Tier 3 Analyst II,2,CHRISTUS Health,descriptionsummarythe information technology t...,"Irving, TX 75038",responsibilities include tier2 tier 3 applicat...,responsibilities include tier2 tier 3 applicat...,No Value,No Value,No Value,https://www.monster.com/job-openings/informati...,|ca|be|me
32146,32146,MRF-Mechanic II,2,MarBorg Recovery,marborg recovery job description job title pla...,"Santa Barbara, CA 93140",responsibilities include limited following per...,responsibilities include limited following per...,5,No Value,|english|spanish,https://www.monster.com/job-openings/mrf-mecha...,|ca|engineering|diploma|be|me|ntc|diploma
32147,32147,Proactive NOC Specialist Tier II,2,New Era Technology,descriptionproactive noc specialist tier iiful...,"Plymouth, MN 55441",descriptionproactive noc specialist tier iiful...,descriptionproactive noc specialist tier iiful...,No Value,No Value,No Value,https://www.monster.com/job-openings/proactive...,|ca|engineering|be|me|msc|itc
32148,32148,"TECHNICIAN, IRRIGATION II (FULL-TIME)",2,SSC Services For Education,posted date aug 2 2021 hiring full-time techni...,"KINGSVILLE, TX 78363",responsibilities repairs existing underground ...,responsibilities repairs existing underground ...,No Value,No Value,No Value,https://www.monster.com/job-openings/technicia...,|ca|m a|be|me


In [107]:
df = pd.read_csv('monster_jobs_scrapped.csv')

In [108]:
educ = []
educate = pd.read_csv('EducationWords.csv')
#educate = pd.read_csv('/content/drive/MyDrive/data/EducationWords.csv') 
for i in educate['Education']:
    educ.append(i)

In [127]:
educ = []
educate = pd.read_csv('EducationWords.csv')
#educate = pd.read_csv('/content/drive/MyDrive/data/EducationWords.csv') 
for i in educate['Education']:
    educ.append(i)

def get_education(x):
    educa =""
    education = []
    for ed in educ:
        if(re.search(ed.lower(),x.lower())):
            educa = ed+" "+educa
            education.append(educa)
            
    if len(education)!=0:
        return (education[-1])
    else:
        return 'No Value'
    
df['Education'] = df['Full Description'].apply(get_education)

In [122]:
get_education(k)

M.D M.Ed M.L ME BE M.A M A CA 


In [153]:
k = df['Responsibilities'][0]

In [154]:
k

'responsibilitiesdesign develop test implement application changesimplement configure manage multiuser systemsimprove automate processes including data retrievalmaintain integrity functionality applicationsdevelop automation scripts programs remove manual system configurationanalyze evaluate propose application changes business sponsors application end usersunderstand details managed system educate end users best practiceswho aredetroit-based rocket companies family businesses providing simple fast trusted digital solutions complex transactions comes flagship business rocket mortgage founded 19'

In [128]:
df['Education'] = df['Full Description'].apply(get_education)

In [129]:
df

,Unnamed: 0,Unnamed: 0.1,Job Title,Searched Job Title,Company Name,Full Description,Location,Skills,Responsibilities,Experience in years,Certification,Foreign Language,Link,Education
0,0,0,Application Engineer,Application Developer,Rocket Auto LLC,minimum qualifications bachelors degree comput...,"Detroit, MI 48226",responsibilitiesdesign develop test implement ...,responsibilitiesdesign develop test implement ...,3,No Value,No Value,https://www.monster.com/job-openings/applicati...,M.D M.Ed M.L ME BE M.A M A CA
1,1,1,Application Developer,Application Developer,Randstad US,job summarythis opportunity mid level applicat...,"Plymouth, MI 48170",responsibilitieswrite requirements test plans ...,responsibilitieswrite requirements test plans ...,No Value,No Value,No Value,https://www.monster.com/job-openings/applicati...,M.D M.L ME B A B.A M.A M. A CA
2,2,2,Sr Windows App Developer,Application Developer,Hypori,us-tx-austincategory information technologyemp...,"Austin, TX 78759",responsibilities participate complete software...,responsibilities participate complete software...,7,No Value,No Value,https://www.monster.com/job-openings/sr-window...,M.D M.Ed M.L ME BE CA
3,3,3,"Application Engineer, Asset Management",Application Developer,Rock Central,preferred qualifications experience taking pro...,"Remote, MI 99999",responsibilities design develop test implement...,responsibilities design develop test implement...,2,No Value,No Value,https://www.monster.com/job-openings/applicati...,M.D M.Ed M.L ME BE B.Com M.A M A CA
4,4,4,COBOL Application Developer,Application Developer,T.E.S. Employment Solution,cobol application developer little rock arkans...,"Little Rock, AR 72201",responsibilitiesfollowing sdlc analysis design...,responsibilitiesfollowing sdlc analysis design...,2,No Value,No Value,https://www.monster.com/job-openings/cobol-app...,M.L ME BE M.A M A CA
...,...,...,...,...,...,...,...,...,...,...,...,...,...,...
32145,32145,32145,Information Technology Tier 3 Analyst II,2,CHRISTUS Health,descriptionsummarythe information technology t...,"Irving, TX 75038",responsibilities include tier2 tier 3 applicat...,responsibilities include tier2 tier 3 applicat...,No Value,No Value,No Value,https://www.monster.com/job-openings/informati...,M.D M.L ME BE M.A M. A CA M. Tech
32146,32146,32146,MRF-Mechanic II,2,MarBorg Recovery,marborg recovery job description job title pla...,"Santa Barbara, CA 93140",responsibilities include limited following per...,responsibilities include limited following per...,5,No Value,|english|spanish,https://www.monster.com/job-openings/mrf-mecha...,M.D B.D.S M.Ed Diploma NTC M.L ME BE M.Sc B.A ...
32147,32147,32147,Proactive NOC Specialist Tier II,2,New Era Technology,descriptionproactive noc specialist tier iiful...,"Plymouth, MN 55441",descriptionproactive noc specialist tier iiful...,descriptionproactive noc specialist tier iiful...,No Value,No Value,No Value,https://www.monster.com/job-openings/proactive...,M.D ITC M.L MSc ME BE B.A Engineering M.A CA
32148,32148,32148,"TECHNICIAN, IRRIGATION II (FULL-TIME)",2,SSC Services For Education,posted date aug 2 2021 hiring full-time techni...,"KINGSVILLE, TX 78363",responsibilities repairs existing underground ...,responsibilities repairs existing underground ...,No Value,No Value,No Value,https://www.monster.com/job-openings/technicia...,M.D M.L ME BE M.Sc B.A M.A M. A M A CA M. Tech


In [130]:
df.to_csv('monster_jobs_scrapped.csv')
#df.to_excel("monster_jobs.xlsx")

In [159]:
l = get_responsibility(k)

In [160]:
l

'No Value'

In [171]:
def get_responsibility(x):
    res = ['responsibility','funcion','activities','activity','responsibilities','responsible','role','roles','day','workday']
    flag = False
    responsibility = []
    x = x.split()
    for word in x:
        print(word)
        for i in res:
            if (re.search(i,word)):
                responsibility.append(word[re.search(res,word).span()[0]:len(word)])
                print(responsibility)
    #return (x[pos:pos+600])

In [210]:
get_responsibility(k)

'following sdlc analysis design development testing implementation maintenancelearning business environment working collaborative team achieve organizational goalsestimates researches documents enhancements problemsability grasp new ideas quickly able thrive dynamic diverse environmen'

In [204]:
k = df['Full Description'][4]

In [211]:
df['Responsibilities'] = df['Responsibilities'].apply(get_responsibility)

In [222]:
def split_(k):
    l = wordninja.split(k)
    x = ''
    for word in l:
        x = x+word+' '
    return (x)

In [223]:
df['Full Description'] = df['Full Description'].apply(split_)

In [224]:
df['Full Description'][0]

'minimum qualifications bachelors degree computer science information technology related field equivalent experience preferred qualifications 3 years experience system administration 3 years programming experience 3 years experience visual basic power shell proficiency microsoft office suite knowledge programming languages net c job summary the application engineer responsible configuring software functions activities web based internally hosted applications responsibilities design develop test implement application changes implement configure manage multiuser systems improve automate processes including data retrieval maintain integrity functionality applications develop automation scripts programs remove manual system configuration analyze evaluate propose application changes business sponsors application end users understand details managed system educate end users best practices who are detroit based rocket companies family businesses providing simple fast trusted digital solutions

In [226]:
df.to_csv('monster_jobs_scrapped.csv')

In [227]:
df['Responsibilities'] = df['Responsibilities'].apply(get_responsibility)

In [233]:
def get_responsibility(x):
    res = ['responsibilities',
           'responsibility',
           'function',
           'activities',
           'activity',
           'responsible',
           'role',
           'roles',
           'day',
           'workday',
           'day to day']
    flag = False
    #x = x.split()
    print(x)
    
    for word in res:
        #print(word)
        try:
            pos = x.index(word)
            flag = True
            return (x[pos+len(word):pos+300])
        except ValueError:
            flag = False
    return ('No Value')

In [230]:
k = df['Full Description'][0]

In [236]:
df['Responsibilities'] = df['Full Description'].apply(get_responsibility)

IOPub data rate exceeded.
The notebook server will temporarily stop sending output
to the client in order to avoid crashing it.
To change this limit, set the config variable
`--NotebookApp.iopub_data_rate_limit`.

Current values:
NotebookApp.iopub_data_rate_limit=1000000.0 (bytes/sec)
NotebookApp.rate_limit_window=3.0 (secs)



In [238]:
df.to_csv('monster_jobs_scrapped.csv')

In [239]:
df

,Unnamed: 0,Unnamed: 0.1,Job Title,Searched Job Title,Company Name,Full Description,Location,Skills,Responsibilities,Experience in years,Certification,Foreign Language,Link,Education
0,0,0,Application Engineer,Application Developer,Rocket Auto LLC,minimum qualifications bachelors degree comput...,"Detroit, MI 48226",responsibilitiesdesign develop test implement ...,design develop test implement application cha...,3,No Value,No Value,https://www.monster.com/job-openings/applicati...,M.D M.Ed M.L ME BE M.A M A CA
1,1,1,Application Developer,Application Developer,Randstad US,job summary this opportunity mid level applica...,"Plymouth, MI 48170",responsibilitieswrite requirements test plans ...,write requirements test plans participates de...,No Value,No Value,No Value,https://www.monster.com/job-openings/applicati...,M.D M.L ME B A B.A M.A M. A CA
2,2,2,Sr Windows App Developer,Application Developer,Hypori,us tx austin category information technology e...,"Austin, TX 78759",responsibilities participate complete software...,participate complete software development pro...,7,No Value,No Value,https://www.monster.com/job-openings/sr-window...,M.D M.Ed M.L ME BE CA
3,3,3,"Application Engineer, Asset Management",Application Developer,Rock Central,preferred qualifications experience taking pro...,"Remote, MI 99999",responsibilities design develop test implement...,design develop test implement mam application...,2,No Value,No Value,https://www.monster.com/job-openings/applicati...,M.D M.Ed M.L ME BE B.Com M.A M A CA
4,4,4,COBOL Application Developer,Application Developer,T.E.S. Employment Solution,cobol application developer little rock arkans...,"Little Rock, AR 72201",responsibilitiesfollowing sdlc analysis design...,following s dlc analysis design development t...,2,No Value,No Value,https://www.monster.com/job-openings/cobol-app...,M.L ME BE M.A M A CA
...,...,...,...,...,...,...,...,...,...,...,...,...,...,...
32145,32145,32145,Information Technology Tier 3 Analyst II,2,CHRISTUS Health,description summary the information technology...,"Irving, TX 75038",responsibilities include tier2 tier 3 applicat...,include tier 2 tier 3 application support med...,No Value,No Value,No Value,https://www.monster.com/job-openings/informati...,M.D M.L ME BE M.A M. A CA M. Tech
32146,32146,32146,MRF-Mechanic II,2,MarBorg Recovery,mar borg recovery job description job title pl...,"Santa Barbara, CA 93140",responsibilities include limited following per...,include limited following performs planned ma...,5,No Value,|english|spanish,https://www.monster.com/job-openings/mrf-mecha...,M.D B.D.S M.Ed Diploma NTC M.L ME BE M.Sc B.A ...
32147,32147,32147,Proactive NOC Specialist Tier II,2,New Era Technology,description proactive noc specialist tier ii f...,"Plymouth, MN 55441",descriptionproactive noc specialist tier iiful...,providing level 2 support systems requirement...,No Value,No Value,No Value,https://www.monster.com/job-openings/proactive...,M.D ITC M.L MSc ME BE B.A Engineering M.A CA
32148,32148,32148,"TECHNICIAN, IRRIGATION II (FULL-TIME)",2,SSC Services For Education,posted date aug 2 2021 hiring full time techni...,"KINGSVILLE, TX 78363",responsibilities repairs existing underground ...,repairs existing underground sprinkler drip s...,No Value,No Value,No Value,https://www.monster.com/job-openings/technicia...,M.D M.L ME BE M.Sc B.A M.A M. A M A CA M. Tech


In [253]:
def get_skills(x):
    res = ['sucessful candidates',
           'skills',
           'expectation',
           'ideal candidate will have',
           'ideal candidate will possess',
           'skills you have',
           'you possess',
           'skill',
           'requirement',
           'you are an ideal candidate if',
           'skills required']
    flag = False
    #x = x.split()
    #print(x)
    
    for word in res:
        #print(word)
        try:
            pos = x.index(word)
            flag = True
            return (x[pos+len(word):pos+300])
        except ValueError:
            flag = False
    return ('No Value')

In [254]:
df['Skills'] = df['Full Description'].apply(get_skills)

In [255]:
df

,Unnamed: 0,Unnamed: 0.1,Job Title,Searched Job Title,Company Name,Full Description,Location,Skills,Responsibilities,Experience in years,Certification,Foreign Language,Link,Education
0,0,0,Application Engineer,Application Developer,Rocket Auto LLC,minimum qualifications bachelors degree comput...,"Detroit, MI 48226",No Value,design develop test implement application cha...,3,No Value,No Value,https://www.monster.com/job-openings/applicati...,M.D M.Ed M.L ME BE M.A M A CA
1,1,1,Application Developer,Application Developer,Randstad US,job summary this opportunity mid level applica...,"Plymouth, MI 48170",good working knowledge technology systems ana...,write requirements test plans participates de...,No Value,No Value,No Value,https://www.monster.com/job-openings/applicati...,M.D M.L ME B A B.A M.A M. A CA
2,2,2,Sr Windows App Developer,Application Developer,Hypori,us tx austin category information technology e...,"Austin, TX 78759",win ui 3 2 net x am lu wp islands audio playb...,participate complete software development pro...,7,No Value,No Value,https://www.monster.com/job-openings/sr-window...,M.D M.Ed M.L ME BE CA
3,3,3,"Application Engineer, Asset Management",Application Developer,Rock Central,preferred qualifications experience taking pro...,"Remote, MI 99999",knowledge experience application administrati...,design develop test implement mam application...,2,No Value,No Value,https://www.monster.com/job-openings/applicati...,M.D M.Ed M.L ME BE B.Com M.A M A CA
4,4,4,COBOL Application Developer,Application Developer,T.E.S. Employment Solution,cobol application developer little rock arkans...,"Little Rock, AR 72201",2 years development experience cobol zo s j c...,following s dlc analysis design development t...,2,No Value,No Value,https://www.monster.com/job-openings/cobol-app...,M.L ME BE M.A M A CA
...,...,...,...,...,...,...,...,...,...,...,...,...,...,...
32145,32145,32145,Information Technology Tier 3 Analyst II,2,CHRISTUS Health,description summary the information technology...,"Irving, TX 75038",maintaining demonstrating good teamwork actio...,include tier 2 tier 3 application support med...,No Value,No Value,No Value,https://www.monster.com/job-openings/informati...,M.D M.L ME BE M.A M. A CA M. Tech
32146,32146,32146,MRF-Mechanic II,2,MarBorg Recovery,mar borg recovery job description job title pl...,"Santa Barbara, CA 93140",work ethic technical aptitude resourcefulness...,include limited following performs planned ma...,5,No Value,|english|spanish,https://www.monster.com/job-openings/mrf-mecha...,M.D B.D.S M.Ed Diploma NTC M.L ME BE M.Sc B.A ...
32147,32147,32147,Proactive NOC Specialist Tier II,2,New Era Technology,description proactive noc specialist tier ii f...,"Plymouth, MN 55441",abilities working knowledge range diagnostic ...,providing level 2 support systems requirement...,No Value,No Value,No Value,https://www.monster.com/job-openings/proactive...,M.D ITC M.L MSc ME BE B.A Engineering M.A CA
32148,32148,32148,"TECHNICIAN, IRRIGATION II (FULL-TIME)",2,SSC Services For Education,posted date aug 2 2021 hiring full time techni...,"KINGSVILLE, TX 78363",calculate proper irrigation regimes audit cal...,repairs existing underground sprinkler drip s...,No Value,No Value,No Value,https://www.monster.com/job-openings/technicia...,M.D M.L ME BE M.Sc B.A M.A M. A M A CA M. Tech


In [256]:
df.to_csv('monster_jobs_scrapped.csv')

In [273]:
certis = {}
for i in df['Job Title']:
    certis[i] = ""

cert = pd.read_csv('C:\\Users\\Dell\\OneDrive\\DataR Labs\\Resume_Parser\\certificates.csv') 
for i in range(0,len(cert)):
    certis[cert['Job'][i].rstrip().lstrip()] = cert['Certificates'][i]

In [280]:
certis = {}
for i in df['Job Title']:
    certis[i] = ""

cert = pd.read_csv('C:\\Users\\Dell\\OneDrive\\DataR Labs\\Resume_Parser\\certificates.csv') 
for i in range(0,len(cert)):
    certis[cert['Job'][i].rstrip().lstrip()] = cert['Certificates'][i]


certifications = []
for ind in range(0,len(df)):
    
    for cer in certis[df['Job Title'][ind]].split(","):
        certitext = ''
        
        if(re.search(cer.lstrip().rstrip(),df['Full Description'][ind])):
            certitext = cer+ " "+certitext 
        
    if(certitext==''):
            certitext= 'No Value'
        
    certifications.append(certitext)
    
df['Certification'] = certifications

In [283]:
df['Certification'] = certifications

In [276]:
df.shape

(32150, 14)

In [285]:
df.replace(' ','No Value')

,Unnamed: 0,Unnamed: 0.1,Job Title,Searched Job Title,Company Name,Full Description,Location,Skills,Responsibilities,Experience in years,Certification,Foreign Language,Link,Education
0,0,0,Application Engineer,Application Developer,Rocket Auto LLC,minimum qualifications bachelors degree comput...,"Detroit, MI 48226",No Value,design develop test implement application cha...,3,No Value,No Value,https://www.monster.com/job-openings/applicati...,M.D M.Ed M.L ME BE M.A M A CA
1,1,1,Application Developer,Application Developer,Randstad US,job summary this opportunity mid level applica...,"Plymouth, MI 48170",good working knowledge technology systems ana...,write requirements test plans participates de...,No Value,No Value,No Value,https://www.monster.com/job-openings/applicati...,M.D M.L ME B A B.A M.A M. A CA
2,2,2,Sr Windows App Developer,Application Developer,Hypori,us tx austin category information technology e...,"Austin, TX 78759",win ui 3 2 net x am lu wp islands audio playb...,participate complete software development pro...,7,No Value,No Value,https://www.monster.com/job-openings/sr-window...,M.D M.Ed M.L ME BE CA
3,3,3,"Application Engineer, Asset Management",Application Developer,Rock Central,preferred qualifications experience taking pro...,"Remote, MI 99999",knowledge experience application administrati...,design develop test implement mam application...,2,No Value,No Value,https://www.monster.com/job-openings/applicati...,M.D M.Ed M.L ME BE B.Com M.A M A CA
4,4,4,COBOL Application Developer,Application Developer,T.E.S. Employment Solution,cobol application developer little rock arkans...,"Little Rock, AR 72201",2 years development experience cobol zo s j c...,following s dlc analysis design development t...,2,No Value,No Value,https://www.monster.com/job-openings/cobol-app...,M.L ME BE M.A M A CA
...,...,...,...,...,...,...,...,...,...,...,...,...,...,...
32145,32145,32145,Information Technology Tier 3 Analyst II,2,CHRISTUS Health,description summary the information technology...,"Irving, TX 75038",maintaining demonstrating good teamwork actio...,include tier 2 tier 3 application support med...,No Value,No Value,No Value,https://www.monster.com/job-openings/informati...,M.D M.L ME BE M.A M. A CA M. Tech
32146,32146,32146,MRF-Mechanic II,2,MarBorg Recovery,mar borg recovery job description job title pl...,"Santa Barbara, CA 93140",work ethic technical aptitude resourcefulness...,include limited following performs planned ma...,5,No Value,|english|spanish,https://www.monster.com/job-openings/mrf-mecha...,M.D B.D.S M.Ed Diploma NTC M.L ME BE M.Sc B.A ...
32147,32147,32147,Proactive NOC Specialist Tier II,2,New Era Technology,description proactive noc specialist tier ii f...,"Plymouth, MN 55441",abilities working knowledge range diagnostic ...,providing level 2 support systems requirement...,No Value,No Value,No Value,https://www.monster.com/job-openings/proactive...,M.D ITC M.L MSc ME BE B.A Engineering M.A CA
32148,32148,32148,"TECHNICIAN, IRRIGATION II (FULL-TIME)",2,SSC Services For Education,posted date aug 2 2021 hiring full time techni...,"KINGSVILLE, TX 78363",calculate proper irrigation regimes audit cal...,repairs existing underground sprinkler drip s...,No Value,No Value,No Value,https://www.monster.com/job-openings/technicia...,M.D M.L ME BE M.Sc B.A M.A M. A M A CA M. Tech


In [316]:
df.to_csv('monster_jobs_scrapped.csv')

In [20]:
def salary(x):
    salary = re.findall('\d+[0-9]{1,9}?( - | to | )\d+[0-9]{1,9} (\w{1,} \w{1,}|\w{1,})',x)
    if salary:
        return salary
    else:
        return ('No Value')

In [21]:
df['Salary'] = df['Full Description'].apply(salary)

In [22]:
df

,Job Title,Searched Job Title,Company Name,Full Description,Location,Job Function,Skills,Responsibilities,Experience in years,Certification,Foreign Language,Link,Education,Salary
0,Network Technician,Network Technician,Interactive Data Systems Limited,1 installing equipment data communications net...,India,Network Technician,s analysis network application design implemen...,No Value,No Value,No Value,No Value,https://www.glassdoor.com/partner/jobListing.h...,M.L ME CA,No Value
1,Network Technician,Network Technician,Interactive Data Systems Limited,1 installing equipment data communications net...,India,No Value,s analysis network application design implemen...,No Value,No Value,No Value,No Value,https://www.glassdoor.com/partner/jobListing.h...,M.L ME CA,No Value
2,Senior Network Technician,Senior Network Technician,Apar Consultancy Services,required skills juniper security network fw pa...,India,Network Technician,juniper security network fw palo alto wan wla...,No Value,No Value,No Value,No Value,No Value,ME,No Value
3,Hardware Networking Specialist,Hardware Networking Specialist,Niyatee Foundation,job description respond hardware related mails...,India,Network Specialist,hardware software networks miscellaneous soft...,No Value,No Value,No Value,No Value,https://www.glassdoor.com/partner/jobListing.h...,B.D.S M.Sc CA,No Value
4,Online Marketer,Online Marketer,Dewelopers,job description optimizing web content seo str...,New Delhi,No Value,No Value,No Value,No Value,No Value,No Value,No Value,M.D B.D.S ME M.A CA,No Value
...,...,...,...,...,...,...,...,...,...,...,...,...,...,...
5897,TGT Teacher,Website Editor,St. Michael's School,job summary st michaels school well establishe...,Rānchī,Teacher,experience teaching cbs e syllabus responsibl...,duties ability create efficient weekly lesson...,No Value,No Value,|english,No Value,M.D B.D.S M.L ME BE M.A CA,No Value
5898,Technical Associate,Website Editor,LeaseBytes Web Solution Private Limited,work experience fresher s job responsibilities...,Rānchī,Technician,required strong computer networking skills go...,troubleshoot day day technical issues challen...,No Value,No Value,No Value,https://www.glassdoor.com/partner/jobListing.h...,M.D ME CA,No Value
5899,TGT Teacher,Website Editor,St. Michael's School,job summary st michaels school well establishe...,Rānchī,Teacher,experience teaching cbs e syllabus responsibl...,duties ability create efficient weekly lesson...,No Value,No Value,|english,No Value,M.D B.D.S M.L ME BE M.A CA,No Value
5900,Network Engineer (2+ Years),2,CSS Corp Private Limited,job profile network engineer location chennai ...,India,Network Engineer,No Value,No Value,4,No Value,No Value,https://www.glassdoor.com/partner/jobListing.h...,B.D.S ME M.A CA,No Value


In [293]:
k = df['Full Description'][1]

In [294]:
k

'job summary this opportunity mid level application developer farmington mi area join established team leading company financial equipment services industry sucessful candidates good working knowledge technology systems analytical problem solving skills location plymouth michigan job type permanent salary 90000 100000 year work hours 8 am 4 pm education bachelors responsibilities write requirements test plans participates development technology solutions meet security requirements interacts vendors business partners problem determination resolution self motivated good problem solving skills support client needs developmental capacity qualifications experience level experienced minimum 4 years experience education bachelors skills c sql web api javascript equal opportunity employer race color religion sex sexual orientation gender identity national origin age genetic information disability protected veteran status legally protected group status '

In [309]:
re.findall('\d+[0-9]{1,9}?( - | to | )\d+[0-9]{1,9} (\w{1,} \w{1,}|\w{1,})',k)

[(' ', 'year work')]

In [314]:
df['Certification'] = df['Certification'].replace(' ','No Value') 

In [315]:
df

,Unnamed: 0,Unnamed: 0.1,Job Title,Searched Job Title,Company Name,Full Description,Location,Skills,Responsibilities,Experience in years,Certification,Foreign Language,Link,Education,Salary
0,0,0,Application Engineer,Application Developer,Rocket Auto LLC,minimum qualifications bachelors degree comput...,"Detroit, MI 48226",No Value,design develop test implement application cha...,3,No Value,No Value,https://www.monster.com/job-openings/applicati...,M.D M.Ed M.L ME BE M.A M A CA,No Value
1,1,1,Application Developer,Application Developer,Randstad US,job summary this opportunity mid level applica...,"Plymouth, MI 48170",good working knowledge technology systems ana...,write requirements test plans participates de...,No Value,No Value,No Value,https://www.monster.com/job-openings/applicati...,M.D M.L ME B A B.A M.A M. A CA,"[( , year work)]"
2,2,2,Sr Windows App Developer,Application Developer,Hypori,us tx austin category information technology e...,"Austin, TX 78759",win ui 3 2 net x am lu wp islands audio playb...,participate complete software development pro...,7,No Value,No Value,https://www.monster.com/job-openings/sr-window...,M.D M.Ed M.L ME BE CA,No Value
3,3,3,"Application Engineer, Asset Management",Application Developer,Rock Central,preferred qualifications experience taking pro...,"Remote, MI 99999",knowledge experience application administrati...,design develop test implement mam application...,2,No Value,No Value,https://www.monster.com/job-openings/applicati...,M.D M.Ed M.L ME BE B.Com M.A M A CA,No Value
4,4,4,COBOL Application Developer,Application Developer,T.E.S. Employment Solution,cobol application developer little rock arkans...,"Little Rock, AR 72201",2 years development experience cobol zo s j c...,following s dlc analysis design development t...,2,No Value,No Value,https://www.monster.com/job-openings/cobol-app...,M.L ME BE M.A M A CA,No Value
...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...
32145,32145,32145,Information Technology Tier 3 Analyst II,2,CHRISTUS Health,description summary the information technology...,"Irving, TX 75038",maintaining demonstrating good teamwork actio...,include tier 2 tier 3 application support med...,No Value,No Value,No Value,https://www.monster.com/job-openings/informati...,M.D M.L ME BE M.A M. A CA M. Tech,No Value
32146,32146,32146,MRF-Mechanic II,2,MarBorg Recovery,mar borg recovery job description job title pl...,"Santa Barbara, CA 93140",work ethic technical aptitude resourcefulness...,include limited following performs planned ma...,5,No Value,|english|spanish,https://www.monster.com/job-openings/mrf-mecha...,M.D B.D.S M.Ed Diploma NTC M.L ME BE M.Sc B.A ...,No Value
32147,32147,32147,Proactive NOC Specialist Tier II,2,New Era Technology,description proactive noc specialist tier ii f...,"Plymouth, MN 55441",abilities working knowledge range diagnostic ...,providing level 2 support systems requirement...,No Value,No Value,No Value,https://www.monster.com/job-openings/proactive...,M.D ITC M.L MSc ME BE B.A Engineering M.A CA,No Value
32148,32148,32148,"TECHNICIAN, IRRIGATION II (FULL-TIME)",2,SSC Services For Education,posted date aug 2 2021 hiring full time techni...,"KINGSVILLE, TX 78363",calculate proper irrigation regimes audit cal...,repairs existing underground sprinkler drip s...,No Value,No Value,No Value,https://www.monster.com/job-openings/technicia...,M.D M.L ME BE M.Sc B.A M.A M. A M A CA M. Tech,"[( , days 125), ( , days employment)]"


In [318]:
df = df.drop(['Unnamed: 0','Unnamed: 0.1'],axis = 1)

In [320]:
df.to_csv('monster_jobs_scrapped.csv')

In [321]:
df.to_excel("monster_jobs.xlsx")

In [382]:
df = pd.read_csv('monster_jobs_scrapped.csv')

In [383]:
df

,Unnamed: 0,Job Title,Searched Job Title,Company Name,Full Description,Location,Skills,Responsibilities,Experience in years,Certification,Foreign Language,Link,Education,Salary
0,0,Application Engineer,Application Developer,Rocket Auto LLC,minimum qualifications bachelors degree comput...,"Detroit, MI 48226",No Value,design develop test implement application cha...,3,No Value,No Value,https://www.monster.com/job-openings/applicati...,M.D M.Ed M.L ME BE M.A M A CA,No Value
1,1,Application Developer,Application Developer,Randstad US,job summary this opportunity mid level applica...,"Plymouth, MI 48170",good working knowledge technology systems ana...,write requirements test plans participates de...,No Value,No Value,No Value,https://www.monster.com/job-openings/applicati...,M.D M.L ME B A B.A M.A M. A CA,"[(' ', 'year work')]"
2,2,Sr Windows App Developer,Application Developer,Hypori,us tx austin category information technology e...,"Austin, TX 78759",win ui 3 2 net x am lu wp islands audio playb...,participate complete software development pro...,7,No Value,No Value,https://www.monster.com/job-openings/sr-window...,M.D M.Ed M.L ME BE CA,No Value
3,3,"Application Engineer, Asset Management",Application Developer,Rock Central,preferred qualifications experience taking pro...,"Remote, MI 99999",knowledge experience application administrati...,design develop test implement mam application...,2,No Value,No Value,https://www.monster.com/job-openings/applicati...,M.D M.Ed M.L ME BE B.Com M.A M A CA,No Value
4,4,COBOL Application Developer,Application Developer,T.E.S. Employment Solution,cobol application developer little rock arkans...,"Little Rock, AR 72201",2 years development experience cobol zo s j c...,following s dlc analysis design development t...,2,No Value,No Value,https://www.monster.com/job-openings/cobol-app...,M.L ME BE M.A M A CA,No Value
...,...,...,...,...,...,...,...,...,...,...,...,...,...,...
32145,32145,Information Technology Tier 3 Analyst II,2,CHRISTUS Health,description summary the information technology...,"Irving, TX 75038",maintaining demonstrating good teamwork actio...,include tier 2 tier 3 application support med...,No Value,No Value,No Value,https://www.monster.com/job-openings/informati...,M.D M.L ME BE M.A M. A CA M. Tech,No Value
32146,32146,MRF-Mechanic II,2,MarBorg Recovery,mar borg recovery job description job title pl...,"Santa Barbara, CA 93140",work ethic technical aptitude resourcefulness...,include limited following performs planned ma...,5,No Value,|english|spanish,https://www.monster.com/job-openings/mrf-mecha...,M.D B.D.S M.Ed Diploma NTC M.L ME BE M.Sc B.A ...,No Value
32147,32147,Proactive NOC Specialist Tier II,2,New Era Technology,description proactive noc specialist tier ii f...,"Plymouth, MN 55441",abilities working knowledge range diagnostic ...,providing level 2 support systems requirement...,No Value,No Value,No Value,https://www.monster.com/job-openings/proactive...,M.D ITC M.L MSc ME BE B.A Engineering M.A CA,No Value
32148,32148,"TECHNICIAN, IRRIGATION II (FULL-TIME)",2,SSC Services For Education,posted date aug 2 2021 hiring full time techni...,"KINGSVILLE, TX 78363",calculate proper irrigation regimes audit cal...,repairs existing underground sprinkler drip s...,No Value,No Value,No Value,https://www.monster.com/job-openings/technicia...,M.D M.L ME BE M.Sc B.A M.A M. A M A CA M. Tech,"[(' ', 'days 125'), (' ', 'days employment')]"


# New

In [2]:
df = pd.read_csv('glassdoor_jobs_scrapped_.csv')

In [3]:
df

,Unnamed: 0,Unnamed: 0.1,Job Title,Searched Job Title,Company Name,Full Description,Location,Job Function,Skills,Responsibilities,Experience in years,Certification,Foreign Language,Link,Education
0,0,0,Network Technician,Network Technician,Interactive Data Systems Limited,1 installing equipment data communications net...,India,Network Technician,1 installing equipment data communications net...,1 installing equipment data communications net...,No Value,No Value,No Value,https://www.glassdoor.com/partner/jobListing.h...,|ca|me
1,1,1,Network Technician,Network Technician,Interactive Data Systems Limited,1 installing equipment data communications net...,India,No Value,1 installing equipment data communications net...,1 installing equipment data communications net...,No Value,No Value,No Value,https://www.glassdoor.com/partner/jobListing.h...,|ca|me
2,2,2,Senior Network Technician,Senior Network Technician,Apar Consultancy Services,required skillsjunipersecuritynetwork fwpalo a...,India,Network Technician,required skillsjunipersecuritynetwork fwpalo a...,required skillsjunipersecuritynetwork fwpalo a...,No Value,No Value,No Value,No Value,|me
3,3,3,Hardware Networking Specialist,Hardware Networking Specialist,Niyatee Foundation,job description respond hardware related mails...,India,Network Specialist,job description respond hardware related mails...,job description respond hardware related mails...,No Value,No Value,No Value,https://www.glassdoor.com/partner/jobListing.h...,|ca
4,4,4,Online Marketer,Online Marketer,Dewelopers,job descriptionoptimizing web content seo stra...,New Delhi,No Value,job descriptionoptimizing web content seo stra...,job descriptionoptimizing web content seo stra...,No Value,No Value,No Value,No Value,|ca|me
...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...
5897,5897,5897,TGT Teacher,Website Editor,St. Michael's School,job summaryst michaels school well-established...,Rānchī,Teacher,responsibilities dutiesability create efficien...,responsibilities dutiesability create efficien...,No Value,No Value,|english,No Value,|ca|be|me
5898,5898,5898,Technical Associate,Website Editor,LeaseBytes Web Solution Private Limited,work experience freshersjob responsibilitiestr...,Rānchī,Technician,responsibilitiestroubleshoot day day technical...,responsibilitiestroubleshoot day day technical...,No Value,No Value,No Value,https://www.glassdoor.com/partner/jobListing.h...,|ca|me
5899,5899,5899,TGT Teacher,Website Editor,St. Michael's School,job summaryst michaels school well-established...,Rānchī,Teacher,responsibilities dutiesability create efficien...,responsibilities dutiesability create efficien...,No Value,No Value,|english,No Value,|ca|be|me
5900,5900,5900,Network Engineer (2+ Years),2,CSS Corp Private Limited,job profile network engineerlocation chennai a...,India,Network Engineer,job profile network engineerlocation chennai a...,job profile network engineerlocation chennai a...,4,No Value,No Value,https://www.glassdoor.com/partner/jobListing.h...,|ca|me


In [4]:
df = df.drop(['Unnamed: 0','Unnamed: 0.1'],axis = 1)

In [5]:
df

,Job Title,Searched Job Title,Company Name,Full Description,Location,Job Function,Skills,Responsibilities,Experience in years,Certification,Foreign Language,Link,Education
0,Network Technician,Network Technician,Interactive Data Systems Limited,1 installing equipment data communications net...,India,Network Technician,1 installing equipment data communications net...,1 installing equipment data communications net...,No Value,No Value,No Value,https://www.glassdoor.com/partner/jobListing.h...,|ca|me
1,Network Technician,Network Technician,Interactive Data Systems Limited,1 installing equipment data communications net...,India,No Value,1 installing equipment data communications net...,1 installing equipment data communications net...,No Value,No Value,No Value,https://www.glassdoor.com/partner/jobListing.h...,|ca|me
2,Senior Network Technician,Senior Network Technician,Apar Consultancy Services,required skillsjunipersecuritynetwork fwpalo a...,India,Network Technician,required skillsjunipersecuritynetwork fwpalo a...,required skillsjunipersecuritynetwork fwpalo a...,No Value,No Value,No Value,No Value,|me
3,Hardware Networking Specialist,Hardware Networking Specialist,Niyatee Foundation,job description respond hardware related mails...,India,Network Specialist,job description respond hardware related mails...,job description respond hardware related mails...,No Value,No Value,No Value,https://www.glassdoor.com/partner/jobListing.h...,|ca
4,Online Marketer,Online Marketer,Dewelopers,job descriptionoptimizing web content seo stra...,New Delhi,No Value,job descriptionoptimizing web content seo stra...,job descriptionoptimizing web content seo stra...,No Value,No Value,No Value,No Value,|ca|me
...,...,...,...,...,...,...,...,...,...,...,...,...,...
5897,TGT Teacher,Website Editor,St. Michael's School,job summaryst michaels school well-established...,Rānchī,Teacher,responsibilities dutiesability create efficien...,responsibilities dutiesability create efficien...,No Value,No Value,|english,No Value,|ca|be|me
5898,Technical Associate,Website Editor,LeaseBytes Web Solution Private Limited,work experience freshersjob responsibilitiestr...,Rānchī,Technician,responsibilitiestroubleshoot day day technical...,responsibilitiestroubleshoot day day technical...,No Value,No Value,No Value,https://www.glassdoor.com/partner/jobListing.h...,|ca|me
5899,TGT Teacher,Website Editor,St. Michael's School,job summaryst michaels school well-established...,Rānchī,Teacher,responsibilities dutiesability create efficien...,responsibilities dutiesability create efficien...,No Value,No Value,|english,No Value,|ca|be|me
5900,Network Engineer (2+ Years),2,CSS Corp Private Limited,job profile network engineerlocation chennai a...,India,Network Engineer,job profile network engineerlocation chennai a...,job profile network engineerlocation chennai a...,4,No Value,No Value,https://www.glassdoor.com/partner/jobListing.h...,|ca|me


In [6]:
import wordninja

def split_(k):
    l = wordninja.split(k)
    x = ''
    for word in l:
        x = x+word+' '
    return (x)

df['Full Description'] = df['Full Description'].apply(split_)

In [9]:
def get_responsibility(x):
    res = ['responsibilities',
           'responsibility',
           'function',
           'activities',
           'activity',
           'responsible',
           'role',
           'roles',
           'day',
           'workday',
           'day to day']
    flag = False
    #x = x.split()
    #print(x)
    
    for word in res:
        #print(word)
        try:
            pos = x.index(word)
            flag = True
            return (x[pos+len(word):pos+300])
        except ValueError:
            flag = False
    return ('No Value')

df['Responsibilities'] = df['Full Description'].apply(get_responsibility)

In [10]:
df

,Job Title,Searched Job Title,Company Name,Full Description,Location,Job Function,Skills,Responsibilities,Experience in years,Certification,Foreign Language,Link,Education
0,Network Technician,Network Technician,Interactive Data Systems Limited,1 installing equipment data communications net...,India,Network Technician,1 installing equipment data communications net...,No Value,No Value,No Value,No Value,https://www.glassdoor.com/partner/jobListing.h...,|ca|me
1,Network Technician,Network Technician,Interactive Data Systems Limited,1 installing equipment data communications net...,India,No Value,1 installing equipment data communications net...,No Value,No Value,No Value,No Value,https://www.glassdoor.com/partner/jobListing.h...,|ca|me
2,Senior Network Technician,Senior Network Technician,Apar Consultancy Services,required skills juniper security network fw pa...,India,Network Technician,required skillsjunipersecuritynetwork fwpalo a...,No Value,No Value,No Value,No Value,No Value,|me
3,Hardware Networking Specialist,Hardware Networking Specialist,Niyatee Foundation,job description respond hardware related mails...,India,Network Specialist,job description respond hardware related mails...,No Value,No Value,No Value,No Value,https://www.glassdoor.com/partner/jobListing.h...,|ca
4,Online Marketer,Online Marketer,Dewelopers,job description optimizing web content seo str...,New Delhi,No Value,job descriptionoptimizing web content seo stra...,No Value,No Value,No Value,No Value,No Value,|ca|me
...,...,...,...,...,...,...,...,...,...,...,...,...,...
5897,TGT Teacher,Website Editor,St. Michael's School,job summary st michaels school well establishe...,Rānchī,Teacher,responsibilities dutiesability create efficien...,duties ability create efficient weekly lesson...,No Value,No Value,|english,No Value,|ca|be|me
5898,Technical Associate,Website Editor,LeaseBytes Web Solution Private Limited,work experience fresher s job responsibilities...,Rānchī,Technician,responsibilitiestroubleshoot day day technical...,troubleshoot day day technical issues challen...,No Value,No Value,No Value,https://www.glassdoor.com/partner/jobListing.h...,|ca|me
5899,TGT Teacher,Website Editor,St. Michael's School,job summary st michaels school well establishe...,Rānchī,Teacher,responsibilities dutiesability create efficien...,duties ability create efficient weekly lesson...,No Value,No Value,|english,No Value,|ca|be|me
5900,Network Engineer (2+ Years),2,CSS Corp Private Limited,job profile network engineer location chennai ...,India,Network Engineer,job profile network engineerlocation chennai a...,No Value,4,No Value,No Value,https://www.glassdoor.com/partner/jobListing.h...,|ca|me


In [11]:
def get_skills(x):
    res = ['sucessful candidates',
           'skills',
           'expectation',
           'ideal candidate will have',
           'ideal candidate will possess',
           'skills you have',
           'you possess',
           'skill',
           'requirement',
           'you are an ideal candidate if',
           'skills required']
    flag = False
    #x = x.split()
    #print(x)
    
    for word in res:
        #print(word)
        try:
            pos = x.index(word)
            flag = True
            return (x[pos+len(word):pos+300])
        except ValueError:
            flag = False
    return ('No Value')

df['Skills'] = df['Full Description'].apply(get_skills)

In [12]:
df

,Job Title,Searched Job Title,Company Name,Full Description,Location,Job Function,Skills,Responsibilities,Experience in years,Certification,Foreign Language,Link,Education
0,Network Technician,Network Technician,Interactive Data Systems Limited,1 installing equipment data communications net...,India,Network Technician,s analysis network application design implemen...,No Value,No Value,No Value,No Value,https://www.glassdoor.com/partner/jobListing.h...,|ca|me
1,Network Technician,Network Technician,Interactive Data Systems Limited,1 installing equipment data communications net...,India,No Value,s analysis network application design implemen...,No Value,No Value,No Value,No Value,https://www.glassdoor.com/partner/jobListing.h...,|ca|me
2,Senior Network Technician,Senior Network Technician,Apar Consultancy Services,required skills juniper security network fw pa...,India,Network Technician,juniper security network fw palo alto wan wla...,No Value,No Value,No Value,No Value,No Value,|me
3,Hardware Networking Specialist,Hardware Networking Specialist,Niyatee Foundation,job description respond hardware related mails...,India,Network Specialist,hardware software networks miscellaneous soft...,No Value,No Value,No Value,No Value,https://www.glassdoor.com/partner/jobListing.h...,|ca
4,Online Marketer,Online Marketer,Dewelopers,job description optimizing web content seo str...,New Delhi,No Value,No Value,No Value,No Value,No Value,No Value,No Value,|ca|me
...,...,...,...,...,...,...,...,...,...,...,...,...,...
5897,TGT Teacher,Website Editor,St. Michael's School,job summary st michaels school well establishe...,Rānchī,Teacher,experience teaching cbs e syllabus responsibl...,duties ability create efficient weekly lesson...,No Value,No Value,|english,No Value,|ca|be|me
5898,Technical Associate,Website Editor,LeaseBytes Web Solution Private Limited,work experience fresher s job responsibilities...,Rānchī,Technician,required strong computer networking skills go...,troubleshoot day day technical issues challen...,No Value,No Value,No Value,https://www.glassdoor.com/partner/jobListing.h...,|ca|me
5899,TGT Teacher,Website Editor,St. Michael's School,job summary st michaels school well establishe...,Rānchī,Teacher,experience teaching cbs e syllabus responsibl...,duties ability create efficient weekly lesson...,No Value,No Value,|english,No Value,|ca|be|me
5900,Network Engineer (2+ Years),2,CSS Corp Private Limited,job profile network engineer location chennai ...,India,Network Engineer,No Value,No Value,4,No Value,No Value,https://www.glassdoor.com/partner/jobListing.h...,|ca|me


In [14]:
import re

certis = {}
for i in df['Job Title']:
    certis[i] = ""

cert = pd.read_csv('C:\\Users\\Dell\\OneDrive\\DataR Labs\\Resume_Parser\\certificates.csv') 
for i in range(0,len(cert)):
    certis[cert['Job'][i].rstrip().lstrip()] = cert['Certificates'][i]


certifications = []
for ind in range(0,len(df)):
    
    for cer in certis[df['Job Title'][ind]].split(","):
        certitext = ''
        
        if(re.search(cer.lstrip().rstrip(),df['Full Description'][ind])):
            certitext = cer+ " "+certitext 
        
    if(certitext==''):
            certitext= 'No Value'
        
    certifications.append(certitext)
    
df['Certification'] = certifications

In [16]:
df = df.replace(' ','No Value')

In [17]:
df

,Job Title,Searched Job Title,Company Name,Full Description,Location,Job Function,Skills,Responsibilities,Experience in years,Certification,Foreign Language,Link,Education
0,Network Technician,Network Technician,Interactive Data Systems Limited,1 installing equipment data communications net...,India,Network Technician,s analysis network application design implemen...,No Value,No Value,No Value,No Value,https://www.glassdoor.com/partner/jobListing.h...,|ca|me
1,Network Technician,Network Technician,Interactive Data Systems Limited,1 installing equipment data communications net...,India,No Value,s analysis network application design implemen...,No Value,No Value,No Value,No Value,https://www.glassdoor.com/partner/jobListing.h...,|ca|me
2,Senior Network Technician,Senior Network Technician,Apar Consultancy Services,required skills juniper security network fw pa...,India,Network Technician,juniper security network fw palo alto wan wla...,No Value,No Value,No Value,No Value,No Value,|me
3,Hardware Networking Specialist,Hardware Networking Specialist,Niyatee Foundation,job description respond hardware related mails...,India,Network Specialist,hardware software networks miscellaneous soft...,No Value,No Value,No Value,No Value,https://www.glassdoor.com/partner/jobListing.h...,|ca
4,Online Marketer,Online Marketer,Dewelopers,job description optimizing web content seo str...,New Delhi,No Value,No Value,No Value,No Value,No Value,No Value,No Value,|ca|me
...,...,...,...,...,...,...,...,...,...,...,...,...,...
5897,TGT Teacher,Website Editor,St. Michael's School,job summary st michaels school well establishe...,Rānchī,Teacher,experience teaching cbs e syllabus responsibl...,duties ability create efficient weekly lesson...,No Value,No Value,|english,No Value,|ca|be|me
5898,Technical Associate,Website Editor,LeaseBytes Web Solution Private Limited,work experience fresher s job responsibilities...,Rānchī,Technician,required strong computer networking skills go...,troubleshoot day day technical issues challen...,No Value,No Value,No Value,https://www.glassdoor.com/partner/jobListing.h...,|ca|me
5899,TGT Teacher,Website Editor,St. Michael's School,job summary st michaels school well establishe...,Rānchī,Teacher,experience teaching cbs e syllabus responsibl...,duties ability create efficient weekly lesson...,No Value,No Value,|english,No Value,|ca|be|me
5900,Network Engineer (2+ Years),2,CSS Corp Private Limited,job profile network engineer location chennai ...,India,Network Engineer,No Value,No Value,4,No Value,No Value,https://www.glassdoor.com/partner/jobListing.h...,|ca|me


In [18]:
educ = []
educate = pd.read_csv('EducationWords.csv')
#educate = pd.read_csv('/content/drive/MyDrive/data/EducationWords.csv') 
for i in educate['Education']:
    educ.append(i)

def get_education(x):
    educa =""
    education = []
    for ed in educ:
        if(re.search(ed.lower(),x.lower())):
            educa = ed+" "+educa
            education.append(educa)
            
    if len(education)!=0:
        return (education[-1])
    else:
        return 'No Value'
    
df['Education'] = df['Full Description'].apply(get_education)

In [19]:
df

,Job Title,Searched Job Title,Company Name,Full Description,Location,Job Function,Skills,Responsibilities,Experience in years,Certification,Foreign Language,Link,Education
0,Network Technician,Network Technician,Interactive Data Systems Limited,1 installing equipment data communications net...,India,Network Technician,s analysis network application design implemen...,No Value,No Value,No Value,No Value,https://www.glassdoor.com/partner/jobListing.h...,M.L ME CA
1,Network Technician,Network Technician,Interactive Data Systems Limited,1 installing equipment data communications net...,India,No Value,s analysis network application design implemen...,No Value,No Value,No Value,No Value,https://www.glassdoor.com/partner/jobListing.h...,M.L ME CA
2,Senior Network Technician,Senior Network Technician,Apar Consultancy Services,required skills juniper security network fw pa...,India,Network Technician,juniper security network fw palo alto wan wla...,No Value,No Value,No Value,No Value,No Value,ME
3,Hardware Networking Specialist,Hardware Networking Specialist,Niyatee Foundation,job description respond hardware related mails...,India,Network Specialist,hardware software networks miscellaneous soft...,No Value,No Value,No Value,No Value,https://www.glassdoor.com/partner/jobListing.h...,B.D.S M.Sc CA
4,Online Marketer,Online Marketer,Dewelopers,job description optimizing web content seo str...,New Delhi,No Value,No Value,No Value,No Value,No Value,No Value,No Value,M.D B.D.S ME M.A CA
...,...,...,...,...,...,...,...,...,...,...,...,...,...
5897,TGT Teacher,Website Editor,St. Michael's School,job summary st michaels school well establishe...,Rānchī,Teacher,experience teaching cbs e syllabus responsibl...,duties ability create efficient weekly lesson...,No Value,No Value,|english,No Value,M.D B.D.S M.L ME BE M.A CA
5898,Technical Associate,Website Editor,LeaseBytes Web Solution Private Limited,work experience fresher s job responsibilities...,Rānchī,Technician,required strong computer networking skills go...,troubleshoot day day technical issues challen...,No Value,No Value,No Value,https://www.glassdoor.com/partner/jobListing.h...,M.D ME CA
5899,TGT Teacher,Website Editor,St. Michael's School,job summary st michaels school well establishe...,Rānchī,Teacher,experience teaching cbs e syllabus responsibl...,duties ability create efficient weekly lesson...,No Value,No Value,|english,No Value,M.D B.D.S M.L ME BE M.A CA
5900,Network Engineer (2+ Years),2,CSS Corp Private Limited,job profile network engineer location chennai ...,India,Network Engineer,No Value,No Value,4,No Value,No Value,https://www.glassdoor.com/partner/jobListing.h...,B.D.S ME M.A CA


In [23]:
df.to_csv('glassdoor_jobs_scrapped.csv')

In [24]:
df = pd.read_csv('monster_jobs_scrapped.csv')

In [25]:
df

,Unnamed: 0,Job Title,Searched Job Title,Company Name,Full Description,Location,Skills,Responsibilities,Experience in years,Certification,Foreign Language,Link,Education,Salary
0,0,Application Engineer,Application Developer,Rocket Auto LLC,minimum qualifications bachelors degree comput...,"Detroit, MI 48226",No Value,design develop test implement application cha...,3,No Value,No Value,https://www.monster.com/job-openings/applicati...,M.D M.Ed M.L ME BE M.A M A CA,No Value
1,1,Application Developer,Application Developer,Randstad US,job summary this opportunity mid level applica...,"Plymouth, MI 48170",good working knowledge technology systems ana...,write requirements test plans participates de...,No Value,No Value,No Value,https://www.monster.com/job-openings/applicati...,M.D M.L ME B A B.A M.A M. A CA,"[(' ', 'year work')]"
2,2,Sr Windows App Developer,Application Developer,Hypori,us tx austin category information technology e...,"Austin, TX 78759",win ui 3 2 net x am lu wp islands audio playb...,participate complete software development pro...,7,No Value,No Value,https://www.monster.com/job-openings/sr-window...,M.D M.Ed M.L ME BE CA,No Value
3,3,"Application Engineer, Asset Management",Application Developer,Rock Central,preferred qualifications experience taking pro...,"Remote, MI 99999",knowledge experience application administrati...,design develop test implement mam application...,2,No Value,No Value,https://www.monster.com/job-openings/applicati...,M.D M.Ed M.L ME BE B.Com M.A M A CA,No Value
4,4,COBOL Application Developer,Application Developer,T.E.S. Employment Solution,cobol application developer little rock arkans...,"Little Rock, AR 72201",2 years development experience cobol zo s j c...,following s dlc analysis design development t...,2,No Value,No Value,https://www.monster.com/job-openings/cobol-app...,M.L ME BE M.A M A CA,No Value
...,...,...,...,...,...,...,...,...,...,...,...,...,...,...
32145,32145,Information Technology Tier 3 Analyst II,2,CHRISTUS Health,description summary the information technology...,"Irving, TX 75038",maintaining demonstrating good teamwork actio...,include tier 2 tier 3 application support med...,No Value,No Value,No Value,https://www.monster.com/job-openings/informati...,M.D M.L ME BE M.A M. A CA M. Tech,No Value
32146,32146,MRF-Mechanic II,2,MarBorg Recovery,mar borg recovery job description job title pl...,"Santa Barbara, CA 93140",work ethic technical aptitude resourcefulness...,include limited following performs planned ma...,5,No Value,|english|spanish,https://www.monster.com/job-openings/mrf-mecha...,M.D B.D.S M.Ed Diploma NTC M.L ME BE M.Sc B.A ...,No Value
32147,32147,Proactive NOC Specialist Tier II,2,New Era Technology,description proactive noc specialist tier ii f...,"Plymouth, MN 55441",abilities working knowledge range diagnostic ...,providing level 2 support systems requirement...,No Value,No Value,No Value,https://www.monster.com/job-openings/proactive...,M.D ITC M.L MSc ME BE B.A Engineering M.A CA,No Value
32148,32148,"TECHNICIAN, IRRIGATION II (FULL-TIME)",2,SSC Services For Education,posted date aug 2 2021 hiring full time techni...,"KINGSVILLE, TX 78363",calculate proper irrigation regimes audit cal...,repairs existing underground sprinkler drip s...,No Value,No Value,No Value,https://www.monster.com/job-openings/technicia...,M.D M.L ME BE M.Sc B.A M.A M. A M A CA M. Tech,"[(' ', 'days 125'), (' ', 'days employment')]"


In [46]:
def exper(x):
    for ex in ['salary','year','years','yrs']:
        if(re.search(ex,x.lower())):
            j = x[re.search(ex,x.lower()).span()[0]-10:re.search(ex,x.lower()).span()[0]]
            #print(j)
            #print(type(j))
            if j.isdigit():
                return j
            else:
                return ("No Value")
    return 'None'

In [34]:
k = df['Full Description'][0]

In [52]:
df['Salary'] = df['Full Description'].apply(exper)

In [48]:
df

,Unnamed: 0,Job Title,Searched Job Title,Company Name,Full Description,Location,Skills,Responsibilities,Experience in years,Certification,Foreign Language,Link,Education,Salary
0,0,Application Engineer,Application Developer,Rocket Auto LLC,minimum qualifications bachelors degree comput...,"Detroit, MI 48226",No Value,design develop test implement application cha...,3,No Value,No Value,https://www.monster.com/job-openings/applicati...,M.D M.Ed M.L ME BE M.A M A CA,No Value
1,1,Application Developer,Application Developer,Randstad US,job summary this opportunity mid level applica...,"Plymouth, MI 48170",good working knowledge technology systems ana...,write requirements test plans participates de...,No Value,No Value,No Value,https://www.monster.com/job-openings/applicati...,M.D M.L ME B A B.A M.A M. A CA,No Value
2,2,Sr Windows App Developer,Application Developer,Hypori,us tx austin category information technology e...,"Austin, TX 78759",win ui 3 2 net x am lu wp islands audio playb...,participate complete software development pro...,7,No Value,No Value,https://www.monster.com/job-openings/sr-window...,M.D M.Ed M.L ME BE CA,No Value
3,3,"Application Engineer, Asset Management",Application Developer,Rock Central,preferred qualifications experience taking pro...,"Remote, MI 99999",knowledge experience application administrati...,design develop test implement mam application...,2,No Value,No Value,https://www.monster.com/job-openings/applicati...,M.D M.Ed M.L ME BE B.Com M.A M A CA,No Value
4,4,COBOL Application Developer,Application Developer,T.E.S. Employment Solution,cobol application developer little rock arkans...,"Little Rock, AR 72201",2 years development experience cobol zo s j c...,following s dlc analysis design development t...,2,No Value,No Value,https://www.monster.com/job-openings/cobol-app...,M.L ME BE M.A M A CA,No Value
...,...,...,...,...,...,...,...,...,...,...,...,...,...,...
32145,32145,Information Technology Tier 3 Analyst II,2,CHRISTUS Health,description summary the information technology...,"Irving, TX 75038",maintaining demonstrating good teamwork actio...,include tier 2 tier 3 application support med...,No Value,No Value,No Value,https://www.monster.com/job-openings/informati...,M.D M.L ME BE M.A M. A CA M. Tech,None
32146,32146,MRF-Mechanic II,2,MarBorg Recovery,mar borg recovery job description job title pl...,"Santa Barbara, CA 93140",work ethic technical aptitude resourcefulness...,include limited following performs planned ma...,5,No Value,|english|spanish,https://www.monster.com/job-openings/mrf-mecha...,M.D B.D.S M.Ed Diploma NTC M.L ME BE M.Sc B.A ...,No Value
32147,32147,Proactive NOC Specialist Tier II,2,New Era Technology,description proactive noc specialist tier ii f...,"Plymouth, MN 55441",abilities working knowledge range diagnostic ...,providing level 2 support systems requirement...,No Value,No Value,No Value,https://www.monster.com/job-openings/proactive...,M.D ITC M.L MSc ME BE B.A Engineering M.A CA,None
32148,32148,"TECHNICIAN, IRRIGATION II (FULL-TIME)",2,SSC Services For Education,posted date aug 2 2021 hiring full time techni...,"KINGSVILLE, TX 78363",calculate proper irrigation regimes audit cal...,repairs existing underground sprinkler drip s...,No Value,No Value,No Value,https://www.monster.com/job-openings/technicia...,M.D M.L ME BE M.Sc B.A M.A M. A M A CA M. Tech,No Value


In [59]:
df.to_csv('glassdoor_jobs_scrapped.csv')

In [60]:
df= pd.read_csv('monster_jobs_scrapped.csv')

In [61]:
df.to_excel("monster_jobs.xlsx")

In [57]:
df = df.fillna('No Value')
df=df.replace('None','No Value')

In [58]:
df

,Unnamed: 0,Job Title,Searched Job Title,Company Name,Full Description,Location,Job Function,Skills,Responsibilities,Experience in years,Certification,Foreign Language,Link,Education,Salary
0,0,Network Technician,Network Technician,Interactive Data Systems Limited,1 installing equipment data communications net...,India,Network Technician,s analysis network application design implemen...,No Value,No Value,No Value,No Value,https://www.glassdoor.com/partner/jobListing.h...,M.L ME CA,No Value
1,1,Network Technician,Network Technician,Interactive Data Systems Limited,1 installing equipment data communications net...,India,No Value,s analysis network application design implemen...,No Value,No Value,No Value,No Value,https://www.glassdoor.com/partner/jobListing.h...,M.L ME CA,No Value
2,2,Senior Network Technician,Senior Network Technician,Apar Consultancy Services,required skills juniper security network fw pa...,India,Network Technician,juniper security network fw palo alto wan wla...,No Value,No Value,No Value,No Value,No Value,ME,No Value
3,3,Hardware Networking Specialist,Hardware Networking Specialist,Niyatee Foundation,job description respond hardware related mails...,India,Network Specialist,hardware software networks miscellaneous soft...,No Value,No Value,No Value,No Value,https://www.glassdoor.com/partner/jobListing.h...,B.D.S M.Sc CA,No Value
4,4,Online Marketer,Online Marketer,Dewelopers,job description optimizing web content seo str...,New Delhi,No Value,No Value,No Value,No Value,No Value,No Value,No Value,M.D B.D.S ME M.A CA,No Value
...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...
5897,5897,TGT Teacher,Website Editor,St. Michael's School,job summary st michaels school well establishe...,Rānchī,Teacher,experience teaching cbs e syllabus responsibl...,duties ability create efficient weekly lesson...,No Value,No Value,|english,No Value,M.D B.D.S M.L ME BE M.A CA,No Value
5898,5898,Technical Associate,Website Editor,LeaseBytes Web Solution Private Limited,work experience fresher s job responsibilities...,Rānchī,Technician,required strong computer networking skills go...,troubleshoot day day technical issues challen...,No Value,No Value,No Value,https://www.glassdoor.com/partner/jobListing.h...,M.D ME CA,No Value
5899,5899,TGT Teacher,Website Editor,St. Michael's School,job summary st michaels school well establishe...,Rānchī,Teacher,experience teaching cbs e syllabus responsibl...,duties ability create efficient weekly lesson...,No Value,No Value,|english,No Value,M.D B.D.S M.L ME BE M.A CA,No Value
5900,5900,Network Engineer (2+ Years),2,CSS Corp Private Limited,job profile network engineer location chennai ...,India,Network Engineer,No Value,No Value,4,No Value,No Value,https://www.glassdoor.com/partner/jobListing.h...,B.D.S ME M.A CA,No Value
